Para poder realizar los ejercicios, iniciamos pyspark y creamos un RDD con cada uno de los archivo CSV  que hay en la practica.

In [1]:
# Iniciamos pyspark y creamos un RDD con cada archivo CSV
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, max, first

spark = SparkSession.builder.getOrCreate()
df21 = spark.read.option("header", "true").csv("Datasets/world-happiness-report-2021.csv")
df = spark.read.option("header", "true").csv("Datasets/world-happiness-report.csv")


# ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)

In [2]:

# Ordenar el DataFrame por puntaje de felicidad de manera descendente
max_ladder_df = df21.sort(desc('Ladder score'))

# Seleccionar el primer país (el más feliz) y su puntaje de felicidad
most_happy_country = max_ladder_df.select('Country name', 'Ladder score').first()
# Creamos un dataframe para mostrarlo con show, no seria necesario porque mostramos los datos en el print
most_happy_country_df = spark.createDataFrame([most_happy_country], ['Country name', 'Ladder score'])

# Mostrar el dataframe y el resultado con el nombre del país y su puntaje de felicidad
most_happy_country_df.show()
print(f"El país más feliz es {most_happy_country[0]} con un puntaje de {most_happy_country[1]}")

+------------+------------+
|Country name|Ladder score|
+------------+------------+
|     Finland|       7.842|
+------------+------------+

El país más feliz es Finland con un puntaje de 7.842


# ¿Cuál es el país más “feliz” del 2021 por continente según la data?

In [9]:
#Este codigo hay que ejecutarlo de forma manual, si se usa el ejecutar automatico para que se ejecuten todas las celdas del notebook no se ejecuta.
#Agrupar por continente, agregar los datos de la max puntuacion y el pais
happiness_continents = df21.groupBy('Regional indicator').agg(max('Ladder score'), first('Country name')) \
    .withColumnRenamed('max(Ladder score)', 'Max score') \
    .withColumnRenamed('first(Country name)', 'Country')

# Ordenar los datos de la max puntuacion de forma desc
most_happy_continents = happiness_continents.sort(desc('Max score')).select('Regional indicator', 'Country', 'Max score')

# Mostrar  los paises mas felices del 2021, truncate es para que no se corte los datos de la tabla y este todo encuadrado.
print(f"Estos son los paises mas felices del 2021 por continente:")
most_happy_continents.show(truncate=False)

Estos son los paises mas felices del 2021 por continente:
+----------------------------------+------------------------+---------+
|Regional indicator                |Country                 |Max score|
+----------------------------------+------------------------+---------+
|Western Europe                    |Finland                 |7.842    |
|North America and ANZ             |New Zealand             |7.277    |
|Middle East and North Africa      |Israel                  |7.157    |
|Latin America and Caribbean       |Costa Rica              |7.069    |
|Central and Eastern Europe        |Czech Republic          |6.965    |
|East Asia                         |Taiwan Province of China|6.584    |
|Southeast Asia                    |Singapore               |6.377    |
|Commonwealth of Independent States|Uzbekistan              |6.179    |
|Sub-Saharan Africa                |Mauritius               |6.049    |
|South Asia                        |Nepal                   |5.269    |
+-----

# ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?

In [4]:
from pyspark.sql.functions import col

# Ordenar los datos por año y Life Ladder en orden descendente
ordered_df = df.orderBy(col("year"), col("Life Ladder").desc())

# Obtener el nombre del país que ocupa el primer lugar en cada año
first_place_df = ordered_df.groupBy("year").agg({"Country name": "first"}).withColumnRenamed("first(Country name)", "Country name")

# Contar cuántas veces cada país ha ocupado el primer lugar, si no pongo limit me aparece el listado de todos los paises que han sido felices y cuantas veces.
count_df = first_place_df.groupBy("Country name").count().orderBy(desc("count")).limit(1)

# Mostrar los resultados
count_df.show()
# Mostrar el resultado con un mensaje
print(f"El país más feliz es {count_df.first()[0]} con un total de {count_df.first()[1]} en el numero 1")

+------------+-----+
|Country name|count|
+------------+-----+
|     Denmark|    7|
+------------+-----+

El país más feliz es Denmark con un total de 7 en el numero 1


# ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

In [5]:
from pyspark.sql.functions import col, lit

# Seleccionamos las columnas que nos interesan del dataframe original, filtramos por el año 2020, y cambiamos el tipo de la columna "Log GDP per capita" a float
df_filtered = df.select("Country name", "year", "Log GDP per capita", "Life Ladder")\
    .filter(col("year") == 2020).withColumn("Log GDP per capita", col("Log GDP per capita").cast("float"))

# Obtenemos el valor máximo de la columna "Log GDP per capita"
max_log_gdp = df_filtered.agg(max("Log GDP per capita")).first()[0]

# Filtramos el dataframe por las filas que tienen el valor máximo en la columna "Log GDP per capita"
df_max_log_gdp = df_filtered.filter(col("Log GDP per capita") == max_log_gdp)

# Contamos el número de países cuyo valor de "Life Ladder" es mayor que el del país con el mayor "Log GDP per capita"
count_higher_ladder = df_filtered.filter(col("Life Ladder") > df_max_log_gdp.select("Life Ladder").first()[0]).count()

# Creamos una nueva columna "Rank" en el dataframe df_max_log_gdp con la funcion lit y añadimos el valor de count_higher_ladder
df_max_log_gdp = df_max_log_gdp.withColumn("Rank", lit(count_higher_ladder + 1))

# Mostramos el resultado
df_max_log_gdp.show()
print(f"El país con el mayor GDP es {df_max_log_gdp.first()[0]} y su posición en felicidad es el puesto {df_max_log_gdp.first()[4]}")


+------------+----+------------------+-----------+----+
|Country name|year|Log GDP per capita|Life Ladder|Rank|
+------------+----+------------------+-----------+----+
|     Ireland|2020|            11.323|      7.035|  13|
+------------+----+------------------+-----------+----+

El país con el mayor GDP es Ireland y su posición en felicidad es el puesto 13


# ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?

In [6]:
from pyspark.sql.functions import mean

"""
Pregunte en discord si se podia ignorar los nulos al ser un tratamiento mas facil de los datos y fue afirmativa.

Cálculo de GDP20 usando la funcion mean que calcula el promedio e ignora a la vez los nulos.
Filtramos el dataframe 'df' para obtener las filas que tienen 'year' igual a 2020,luego aplicamos
la función 'mean' a la columna 'Log GDP per capita', y finalmente convertimos el resultado a una RDD
para obtener el primer elemento de la tupla y su valor
"""
GDP20 = df.filter(col("year") == 2020).agg(mean("Log GDP per capita")).rdd.first()[0]

# Cálculo de GDP21 usando la funcion mean que calcula el promedio e ignora a la vez los nulos
GDP21 = df21.agg(mean("Logged GDP per capita")).rdd.first()[0]

# Cálculo de la variación
percentage = ((GDP21 - GDP20) / GDP20) * 100

print(round(percentage, 2))
print("Al ser un resultado negativo podemos decir que disminuyó.")


-3.27
Al ser un resultado negativo podemos decir que disminuyó.


# ¿Cuál es el país con mayor expectativa de vida (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?

In [7]:
from pyspark.sql.functions import count

# Obtener la mayor expectativa de vida en cada año
max_life = df.groupBy("year").agg(max("Healthy life expectancy at birth").alias("max_exp_life"))

# Unir con el dataframe original para obtener la fila correspondiente al país con la mayor expectativa de vida en cada año
max_life = df.join(max_life, ["year"]).filter(
    df["Healthy life expectancy at birth"] == max_life["max_exp_life"])\
    .select(df["year"], df["Country name"], df["Healthy life expectancy at birth"])

# Contar las veces que aparece cada país en el resultado y mostrar el que más tiene
most_country = max_life.groupBy("Country name").agg(count("*").alias("count")).orderBy(desc("count"))\
    .select("Country name", "count").first()

# Mostrar los resultados ordenados por año.
max_life.orderBy(("year")).show()
print(f"El país que más veces ha tenido la mejor esperanza de vida es {most_country[0]} con {most_country[1]} años siendo el 1º")

+----+------------+--------------------------------+
|year|Country name|Healthy life expectancy at birth|
+----+------------+--------------------------------+
|2005|       Japan|                          73.200|
|2006|   Singapore|                          73.600|
|2007|   Singapore|                          73.900|
|2008|   Singapore|                          74.200|
|2009|   Singapore|                          74.500|
|2010|   Singapore|                          74.800|
|2011|   Singapore|                          75.020|
|2012|       Japan|                          74.160|
|2013|   Singapore|                          75.460|
|2014|   Singapore|                          75.680|
|2015|   Singapore|                          75.900|
|2016|   Singapore|                          76.200|
|2017|   Singapore|                          76.500|
|2018|   Singapore|                          76.800|
|2019|   Singapore|                          77.100|
|2020|       Japan|                          7

In [8]:

# Buscar en 2019 que esperanza de vida tenia el pais con mejor esperanza de vida.
max2019 = df.filter((col("year") == 2019) & (col("Country name") == most_country[0]))

# Mostrar el resultado
max2019.select("year", "Country name","Healthy life expectancy at birth").show()
print(f"En 2019 {most_country[0]} tenia una esperanza de vida de {most_country[1]} ")



+----+------------+--------------------------------+
|year|Country name|Healthy life expectancy at birth|
+----+------------+--------------------------------+
|2019|   Singapore|                          77.100|
+----+------------+--------------------------------+

En 2019 Singapore tenia una esperanza de vida de 13 
